In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

from peft import PeftModel
import faiss
import pickle


# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

In [7]:
import os

# 특정 폴더 지정 (현재 폴더: ".")
folder_path = "./data/"

# 특정 단어(prefix)로 시작하는 파일만 필터링
prefix = "외국인근로자 고용·취업"  # 예: 'data'로 시작하는 파일만 찾기
files = [f for f in os.listdir(folder_path) if f.startswith(prefix)]

print(files)
print(len(files))

['외국인근로자 고용·취업 _ 외국인근로자 고용 _ 사용자 준수사항 _ 그 밖의 준수사항.pdf', '외국인근로자 고용·취업 _ 외국인근로자 고용 _ 사용자 준수사항 _ 보험 가입 의무.pdf', '외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 출·입국 및 체류 _ 체류 관련 준수사항.pdf', '외국인근로자 고용·취업 _ 외국인근로자 고용·취업 개요 _ 외국인근로자 권리구제 제도 _ 노동 관련 권리구제.pdf', '외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 여성근로자 보호 _ 직장 내 성희롱 예방 및 조치.pdf', '외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 준수사항 _ 보험가입 의무.pdf', '외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용자격·범위 _ 사용자 자격요건.pdf', '외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 출·입국 및 체류 _ 출입국 관련 절차.pdf', '외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 국제결혼·자녀출생 및 사망 _ 장례 관련 절차.pdf', '외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업절차 _ 방문취업(H-2) 체류자격자 취업절차.pdf', '외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 국제결혼·자녀출생 및 사망 _ 자녀 출생·취학 관련 절차.pdf', '외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용제한 _ 고용허가 또는 특례고용확인의 취소 및 고용 제한.pdf', '외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용자격·범위 _ 외국인근로자 도입규모 및 업종.pdf', '외국인근로자 고용·취업 _ 외국인근로자 고용·취업 개요 _ 외국인근로자 권리보호 제도 _ 사회보장 관계 법령에 의한 보호.pdf', '외국인근로자 고용·취업 _ 외국인근로자 고용·취업 개요 _ 외국인근로자 고용·취업 제도 _ 「외국인근로자의 고용 등에 관한 법률」의 외국인근로자 고용·취업.pdf', '외국인근로자 고용·취업 _ 외국인

In [14]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [16]:
import os
import fitz
import faiss
import pickle
import unicodedata
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

# PDF 처리 함수
def process_pdf(file_path, chunk_size=768, chunk_overlap=64):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    doc = fitz.open(file_path)
    text = ''

    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()

    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)

    # Document 객체 리스트 생성
    chunks = [Document(page_content=t, metadata={"source": file_path}) for t in chunk_temp]
    return chunks


# FAISS DB 생성 함수
def create_vector_db(chunks, model_path="nlpai-lab/KURE-v1"):
    """FAISS DB 생성"""
    model_kwargs = {'device': 'cuda:1'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db


# 카테고리별 FAISS DB 통합
def process_pdfs_by_category(base_directory, category_mapping):
    """카테고리별로 PDF를 처리하여 FAISS DB를 하나로 통합"""
    pdf_databases = {}

    for category, pdf_files in category_mapping.items():
        category_dir = os.path.join(base_directory, category)
        os.makedirs(category_dir, exist_ok=True)

        all_chunks = []  # 해당 카테고리의 모든 PDF chunks 저장

        for pdf_name in tqdm(pdf_files, desc=f"Processing PDFs for {category}"):
            full_path = os.path.join(base_directory, pdf_name)
            pdf_title = os.path.splitext(pdf_name)[0]
            print(f"Processing {pdf_title} in category {category}...")

            # PDF 처리 후 chunk 생성
            chunks = process_pdf(full_path)

            # 빈 chunks 처리
            if not chunks:
                print(f"Skipping {pdf_title} due to empty content.")
                continue

            # 모든 chunks를 하나의 리스트에 추가
            all_chunks.extend(chunks)

        # 카테고리 내 전체 PDF의 chunks를 하나의 FAISS DB로 통합
        if all_chunks:
            db = create_vector_db(all_chunks)

            # Retriever 생성
            retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 10})

            # FAISS DB 저장
            faiss_path = os.path.join(category_dir, f"{category}_faiss_db.index")
            faiss.write_index(db.index, faiss_path)

            # 매핑 객체 저장
            with open(os.path.join(category_dir, f"{category}_index_to_docstore_id.pkl"), "wb") as f:
                pickle.dump(db.index_to_docstore_id, f)

            with open(os.path.join(category_dir, f"{category}_docstore.pkl"), "wb") as f:
                pickle.dump(db.docstore, f)

            # 결과 저장
            pdf_databases[category] = {
                'db': db,
                'retriever': retriever
            }

    return pdf_databases

category_mapping = {
    "고용절차": [
        "외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용자격·범위 _ 사용자 자격요건.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용자격·범위 _ 외국인근로자 도입규모 및 업종.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용절차 _ 방문취업(H-2) 체류자격자 고용절차.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용절차 _ 비전문취업(E-9) 체류자격자 고용절차.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용제한 _ 고용허가 또는 특례고용확인의 취소 및 고용 제한.pdf"
    ],
    "취업 및 준수사항": [
        "외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업절차 _ 방문취업(H-2) 체류자격자 취업절차.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업절차 _ 비전문취업(E-9) 체류자격자 취업절차.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업자격 _ 외국인근로자 자격요건.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 준수사항 _ 보험가입 의무.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 준수사항 _ 「출입국관리법」의 준수사항.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업 지원 및 제한 _ 취업 기간 및 사업장 제한.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업 지원 및 제한 _ 고충상담 및 처리 지원.pdf"
    ],
    "노동법 및 권리보호": [
        "외국인근로자 고용·취업 _ 외국인근로자 고용·취업 개요 _ 외국인근로자 권리구제 제도 _ 노동 관련 권리구제.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용·취업 개요 _ 외국인근로자 권리구제 제도 _ 사회보장 관련 권리구제.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용·취업 개요 _ 외국인근로자 권리보호 제도 _ 노동 관계 법령에 의한 보호.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용·취업 개요 _ 외국인근로자 권리보호 제도 _ 사회보장 관계 법령에 의한 보호.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용·취업 개요 _ 외국인근로자 고용·취업 제도 _ 「외국인근로자의 고용 등에 관한 법률」의 외국인근로자 고용·취업.pdf"
    ],
    "출입국 및 체류": [
        "외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 출·입국 및 체류 _ 출입국 관련 절차.pdf",
        "외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 출·입국 및 체류 _ 체류 관련 준수사항.pdf",
        "외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 국제결혼·자녀출생 및 사망 _ 국제결혼 관련 절차.pdf",
        "외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 국제결혼·자녀출생 및 사망 _ 자녀 출생·취학 관련 절차.pdf",
        "외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 국제결혼·자녀출생 및 사망 _ 장례 관련 절차.pdf"
    ],
    "사용자 의무": [
        "외국인근로자 고용·취업 _ 외국인근로자 고용 _ 사용자 준수사항 _ 그 밖의 준수사항.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용 _ 사용자 준수사항 _ 보험 가입 의무.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용 _ 사용자 준수사항 _ 고용변동 등 신고 의무.pdf",
        "외국인근로자 고용·취업 _ 외국인근로자 고용 _ 사용자 준수사항 _ 임금 및 근로조건 사항.pdf",
        "외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 여성근로자 보호 _ 직장 내 성희롱 예방 및 조치.pdf",
        "외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 여성근로자 보호 _ 여성보호를 위한 근로기준.pdf"
    ]
}

# 실행
base_directory = "./data/raw"
process_pdfs_by_category(base_directory, category_mapping)


Processing PDFs for 고용절차:   0%|          | 0/5 [00:00<?, ?it/s]

Processing PDFs for 고용절차: 100%|██████████| 5/5 [00:00<00:00, 31.92it/s]


Processing 외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용자격·범위 _ 사용자 자격요건 in category 고용절차...
Processing 외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용자격·범위 _ 외국인근로자 도입규모 및 업종 in category 고용절차...
Processing 외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용절차 _ 방문취업(H-2) 체류자격자 고용절차 in category 고용절차...
Processing 외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용절차 _ 비전문취업(E-9) 체류자격자 고용절차 in category 고용절차...
Processing 외국인근로자 고용·취업 _ 외국인근로자 고용 _ 외국인근로자 고용제한 _ 고용허가 또는 특례고용확인의 취소 및 고용 제한 in category 고용절차...


Processing PDFs for 취업 및 준수사항: 100%|██████████| 7/7 [00:00<00:00, 46.74it/s]


Processing 외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업절차 _ 방문취업(H-2) 체류자격자 취업절차 in category 취업 및 준수사항...
Processing 외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업절차 _ 비전문취업(E-9) 체류자격자 취업절차 in category 취업 및 준수사항...
Processing 외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업자격 _ 외국인근로자 자격요건 in category 취업 및 준수사항...
Processing 외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 준수사항 _ 보험가입 의무 in category 취업 및 준수사항...
Processing 외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 준수사항 _ 「출입국관리법」의 준수사항 in category 취업 및 준수사항...
Processing 외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업 지원 및 제한 _ 취업 기간 및 사업장 제한 in category 취업 및 준수사항...
Processing 외국인근로자 고용·취업 _ 외국인근로자 취업 _ 외국인근로자 취업 지원 및 제한 _ 고충상담 및 처리 지원 in category 취업 및 준수사항...


...


Processing PDFs for 출입국 및 체류: 100%|██████████| 5/5 [00:00<00:00, 52.48it/s]


Processing 외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 출·입국 및 체류 _ 출입국 관련 절차 in category 출입국 및 체류...
Processing 외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 출·입국 및 체류 _ 체류 관련 준수사항 in category 출입국 및 체류...
Processing 외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 국제결혼·자녀출생 및 사망 _ 국제결혼 관련 절차 in category 출입국 및 체류...
Processing 외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 국제결혼·자녀출생 및 사망 _ 자녀 출생·취학 관련 절차 in category 출입국 및 체류...
Processing 외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 국제결혼·자녀출생 및 사망 _ 장례 관련 절차 in category 출입국 및 체류...


Processing PDFs for 사용자 의무: 100%|██████████| 6/6 [00:00<00:00, 67.17it/s]


Processing 외국인근로자 고용·취업 _ 외국인근로자 고용 _ 사용자 준수사항 _ 그 밖의 준수사항 in category 사용자 의무...
Processing 외국인근로자 고용·취업 _ 외국인근로자 고용 _ 사용자 준수사항 _ 보험 가입 의무 in category 사용자 의무...
Processing 외국인근로자 고용·취업 _ 외국인근로자 고용 _ 사용자 준수사항 _ 고용변동 등 신고 의무 in category 사용자 의무...
Processing 외국인근로자 고용·취업 _ 외국인근로자 고용 _ 사용자 준수사항 _ 임금 및 근로조건 사항 in category 사용자 의무...
Processing 외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 여성근로자 보호 _ 직장 내 성희롱 예방 및 조치 in category 사용자 의무...
Processing 외국인근로자 고용·취업 _ 국내체류 관련 정보 _ 여성근로자 보호 _ 여성보호를 위한 근로기준 in category 사용자 의무...


{'고용절차': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f2566bb6de0>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f2566bb6de0>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 10})},
 '취업 및 준수사항': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f256559aba0>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f256559aba0>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 10})},
 '노동법 및 권리보호': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f25655a89b0>,
  'retriever': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f25655a89b0>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 10})},
 '출입국 및 체류': {'db': <langchain_community.vectorstores.faiss.FAISS at 0x7f2552